In [15]:
import os 
import plotly.graph_objects as go

In [16]:
def read_file(src):
    ch1_file = open(os.path.join(src))

    ch1 = ch1_file.read()

    ch1_file.close()
    
    ch1_lines = ch1.strip().split('\n')

    parsed_data = {}

    for i in range(23):
        key,value = ch1_lines[i].strip().split(',')[:2]
        parsed_data[key] = value
    
    volt = []

    for i in ch1_lines[25:]:
        try:
            volt.append(float(i.strip().split(',')[0]))        
        except:
            print('bad parsing data'+i)
        
    parsed_data['volt'] = volt
    return (parsed_data)

''' 
def mapping(data,y1,y2):
    maximum = max(data)
    minimum = min(data)
    interval_x = maximum-minimum
    interval_y = abs(y1-y2)
    result = []
    for i in data:
        result.append(i/interval_x*interval_y)
    return (result)
'''

def find_peaks(volt):
    peak_volt = max(volt)
    # This will storaged the result of the time where peak occured
    peak_position = []
    # TMP storage the index each group when peak value occurs
    # you can also see this as the time of each peak in group
    # since the time will be index * sample_period
    TMP = []
    # Flag used for determine the group 
    # When False and deteced peak voltage occurs 
    # it let program start to record a new group of peak point
    flag = False
    # target used to storage the first peak point in each peak 
    target = 0
    threshold = 500
    for i in range(len(volt)):
        if (volt[i] == peak_volt):
            target = i
            flag = True
            TMP.append(i)
        if (flag == True and (i>target+threshold or i==len(volt)-1)):
            # if exceed the threshold turn off the record ($flag) 
            # and push the averaged result from TMP to $peak_position
            # clear the TMP for next group
            peak_position.append(sum(TMP)//len(TMP))
            TMP = [] 
            flag = False
    return(peak_position)

def optimized_find_peaks(volt):
    peak_volt = max(volt)
    n = 0
    for i in volt:
        if i==peak_volt:
            n+=1
    if n<50:
        filter_list = set(volt)
        filter_list.remove(max(filter_list))
        peak_volt = max(filter_list)
    peak_position = []

    TMP = []

    flag = False
    target = 0
    threshold = 500
    for i in range(len(volt)):
        if (volt[i] == peak_volt):
            target = i
            flag = True
            TMP.append(i)
        if (flag == True and (i>target+threshold or i==len(volt)-1)):
            # if exceed the threshold turn off the record ($flag) 
            # and push the averaged result from TMP to $peak_position
            # clear the TMP for next group
            peak_position.append(sum(TMP)//len(TMP))
            TMP = [] 
            flag = False
    return(peak_position)

def get_label_string(magnitude):
    if (magnitude<1):
        return(str(magnitude*1000)+' mV')
    else:
        return(str(magnitude)+' V')



# Input source and capacitor's volt waveform 

In [46]:


src = '.\secound_exp\ALL0003' 


ch1 = read_file(os.path.join(src,'1.CSV')) 
ch2 = read_file(os.path.join(src,'2.CSV')) 

ch1_div = float(ch1['Vertical Scale'])
ch2_div = float(ch2['Vertical Scale'])

ch1_sample_period = float(ch1['Sampling Period'])
ch2_sample_period = float(ch2['Sampling Period'])

ch1_mapped_volt = [i*ch1_div/32 for i in ch1['volt']]
ch2_mapped_volt = [i*ch2_div/32 for i in ch2['volt']]

ch1_reveal_div = 1
ch2_reveal_div = 1

ch1_trace = go.Scattergl(
    x=[i*ch1_sample_period for i in range(len(ch1['volt']))],
    y=[i/ch1_reveal_div for i in ch1_mapped_volt],
    mode='markers',
    name= 'Input Source        (div : '+get_label_string(ch1_reveal_div)+')' ,
)

ch2_trace = go.Scattergl(
    x=[i*ch2_sample_period for i in range(len(ch2['volt']))],
    y=[i/ch2_reveal_div for i in ch2_mapped_volt],
    mode='markers',
    name= 'Capacitor Voltage (div : '+get_label_string(ch2_reveal_div) +')',
)

fig = go.Figure()
fig.add_trace(ch1_trace)
fig.add_trace(ch2_trace)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01,
    font = dict(size=12)
))

Xaxis_title = "Time"
Yaxis_title = "Volt / Div"
fig['layout'].update(width=960,height=720,xaxis_title=Xaxis_title,
                        yaxis_title=Yaxis_title,
                        font=dict(family='Microsoft JhengHei', size=18,), xaxis=dict(tickangle=0)
                        )

fig.write_html('RC_input_vs_capacitor.html', auto_open=True)


# Input source and resistor's volt waveform 

In [44]:
import os 
import plotly.graph_objects as go

src = '.\secound_exp\ALL0004' 


ch1 = read_file(os.path.join(src,'1.CSV')) 
ch2 = read_file(os.path.join(src,'2.CSV')) 

ch1_div = float(ch1['Vertical Scale'])
ch2_div = float(ch2['Vertical Scale'])

ch1_sample_period = float(ch1['Sampling Period'])
ch2_sample_period = float(ch2['Sampling Period'])

ch1_mapped_volt = [i*ch1_div/32 for i in ch1['volt']]
ch2_mapped_volt = [i*ch2_div/32 for i in ch2['volt']]

ch1_reveal_div = 1
ch2_reveal_div = 1

ch1_trace = go.Scattergl(
    x=[i*ch1_sample_period for i in range(len(ch1['volt']))],
    y=[i/ch1_reveal_div for i in ch1_mapped_volt],

    mode='markers',
    name= 'Input Source        (div : '+get_label_string(ch1_reveal_div)+')' ,
)

ch2_trace = go.Scattergl(
    x=[i*ch2_sample_period for i in range(len(ch2['volt']))],
    y=[i/ch2_reveal_div for i in ch2_mapped_volt],

    mode='markers',
    name= 'Resistor Voltage (div : '+get_label_string(ch2_reveal_div) +')',
)

fig = go.Figure()
fig.add_trace(ch1_trace)
fig.add_trace(ch2_trace)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01,
    font = dict(size=12)
))

Xaxis_title = "Time"
Yaxis_title = "Volt / Div"
fig['layout'].update( width=960,height=720,xaxis_title=Xaxis_title,
                        yaxis_title=Yaxis_title,
                        font=dict(family='Microsoft JhengHei', size=18,), xaxis=dict(tickangle=0)
                        )

fig.write_html('RC_input_vs_resistor.html', auto_open=True)


# Calculate Phase shift

In [7]:
def get_shift(main,slave):
    if (len(main)!=len(slave)):
        print("parse error")
        return -1
    avg_shift = 0
    tmp = []
    for i in range(len(main)-1):
        tmp.append(abs(main[i+1]-main[i]))
        tmp.append(abs(slave[i+1]-slave[i]))
    print('periods : ' ,end='')
    print(tmp)
    avg_period = sum(tmp)/len(tmp)
    tmp = []
    for i in range(len(main)):
        tmp.append(slave[i]-main[i])
    print('shifts : ' ,end='')
    print(tmp)
    avg_shift = sum(tmp)/len(tmp)

    return(avg_shift/avg_period*360)

In [8]:
src = '.\secound_exp\ALL0003' 


ch1 = read_file(os.path.join(src,'1.CSV')) 
ch2 = read_file(os.path.join(src,'2.CSV')) 

ch1_div = float(ch1['Vertical Scale'])
ch2_div = float(ch2['Vertical Scale'])

ch1_mapped_volt = [i*ch1_div/32 for i in ch1['volt']]
ch2_mapped_volt = [i*ch2_div/32 for i in ch2['volt']]

ch1_peaks = find_peaks(ch1['volt']) 
ch2_peaks = optimized_find_peaks(ch2['volt']) 

print(ch1_peaks)
print(ch2_peaks)

get_shift(ch1_peaks,ch2_peaks)

[2487, 5613, 8739]
[2705, 5827, 8950]
periods : [3126, 3122, 3126, 3123]
shifts : [218, 214, 211]


24.69712731055453

In [10]:
src = '.\secound_exp\ALL0004' 


ch1 = read_file(os.path.join(src,'1.CSV')) 
ch2 = read_file(os.path.join(src,'2.CSV')) 

ch1_div = float(ch1['Vertical Scale'])
ch2_div = float(ch2['Vertical Scale'])

ch1_mapped_volt = [i*ch1_div/32 for i in ch1['volt']]
ch2_mapped_volt = [i*ch2_div/32 for i in ch2['volt']]

ch1_peaks = find_peaks(ch1['volt']) 
ch2_peaks = optimized_find_peaks(ch2['volt']) 

print(ch1_peaks)
print(ch2_peaks)

# A redundant peak detected so cut the $ch1_peaks from [1:]
get_shift(ch1_peaks[1:],ch2_peaks)

[160, 3286, 6408, 9533]
[2714, 5840, 8964]
periods : [3122, 3126, 3125, 3124]
shifts : [-572, -568, -569]


-65.64135392494198